In [1]:
import time
import pytz
import pandas as pd
import base58
import datetime
from solana.rpc.api import Client
from solders.pubkey import Pubkey

pd.options.display.max_rows = 500

In [2]:
class TransactionTypes:
    vote = "VOTE"
    sell = "SELL"
    transfer = "TRANSFER"
    reimburse = "REIMBURSE"
    rewards = "REWARDS"

class TransactionLine:
    def __init__(self, pubkey, tx, tx_detail):
        self.pubkey = pubkey
        self.tx = tx
        self.tx_detail = tx_detail
    
    @property
    def time(self):
        dt_object = datetime.datetime.fromtimestamp(self.tx.block_time, pytz.utc)
        pst_timezone = pytz.timezone('US/Pacific')
        pst_time = dt_object.astimezone(pst_timezone)
        return pst_time.strftime('%Y-%m-%d %I:%M:%S %p')
    
    @property
    def _pre_balances(self):
        return [bal / 1e9 for bal in self.tx_detail.value.transaction.meta.pre_balances]

    @property
    def _post_balances(self):
        return [bal / 1e9 for bal in self.tx_detail.value.transaction.meta.post_balances]

    @property
    def _account_keys(self):
        return [key.__str__() for key in tx_details.value.transaction.transaction.message.account_keys]
    
    @property
    def _self_index(self):
        for i, key in enumerate(self._account_keys):
            if key == self.pubkey:
                return i
    
    @property
    def balance(self):
        return self._post_balances[self._self_index]

    @property
    def amount(self):
        index = self._self_index
        return self._post_balances[index] - self._pre_balances[index]
    
    @property
    def type(self):
        vote_program_id = "Vote111111111111111111111111111111111111111"
        if vote_program_id in self._account_keys:
            return TransactionTypes.vote

    @property
    def to_dict(self):
        return {
            'time': self.time,
            'amount': self.amount,
            'balance': self.balance,
            'type': self.type,
        }
        

In [3]:
# client = Client("https://api.mainnet-beta.solana.com")
client = Client("http://127.0.0.1:8899")

wallet_address = "NLMSHTjmSiRxGJPs3uaqtsFBC2dTGYwK41U18Nmw5kH"

def get_transactions(wallet_address, before):
    start = datetime.datetime.now()
    if before:
        print(f"Getting transactions from before {str(before)[:3]}...{str(before)[-3:]}: ", end='')
    decoded_address = base58.b58decode(wallet_address)
    ret = client.get_signatures_for_address(
        account=Pubkey(decoded_address), 
        limit=1000,
        before=before,
    )
    print(datetime.datetime.now() - start)
    return ret
    


In [4]:
lines = []
before = None

while len(lines) < 10000:
    tries = 0
    transactions = None
    while tries < 4:
        transactions = get_transactions(wallet_address, before=before).value
        if transactions is None or len(transactions) == 0:
            print('no transactions')
            tries += 1
            time.sleep(30)
        else:
            break

    if transactions is None or len(transactions) == 0:
        print('Failed to fetch transactions after 4 retries')
        break

    for transaction in transactions:
        tx_details = client.get_transaction(transaction.signature)
        lines.append(TransactionLine(wallet_address, transaction, tx_details).to_dict)
    
    if transactions:
        before = transactions[-1].signature

0:00:00.106983
Getting transactions from before gR7...t5W: 0:00:00.131902
Getting transactions from before 5Kd...tYU: 0:00:00.082915
Getting transactions from before 3BY...Bt2: 0:00:00.054183
no transactions
Getting transactions from before 3BY...Bt2: 0:00:00.001742
no transactions
Getting transactions from before 3BY...Bt2: 0:00:00.001766
no transactions
Getting transactions from before 3BY...Bt2: 0:00:00.001490
no transactions
Failed to fetch transactions after 4 retries


In [5]:
pd.DataFrame(lines)

,time,amount,balance,type
0,2025-01-27 06:27:20 PM,-0.000005,6.613310,VOTE
1,2025-01-27 06:27:20 PM,-0.000005,6.613315,VOTE
2,2025-01-27 06:27:20 PM,-0.000005,6.613320,VOTE
3,2025-01-27 06:27:19 PM,-0.000005,6.613325,VOTE
4,2025-01-27 06:27:19 PM,-0.000005,6.613330,VOTE
...,...,...,...,...
2600,2025-01-27 06:09:42 PM,-0.000005,6.171834,VOTE
2601,2025-01-27 06:09:41 PM,-0.000005,6.171839,VOTE
2602,2025-01-27 06:09:41 PM,-0.000005,6.171844,VOTE
2603,2025-01-27 06:09:41 PM,-0.000005,6.171849,VOTE


In [100]:
len(lines)

2606